In [1]:
import torch

if torch.cuda.is_available():
    print('Yes')
else:
    print('No')

Yes


In [10]:
from time import time

import cv2
import cvzone
from cvzone.FaceDetectionModule import FaceDetector

####################################
classID = 3  # ['device', 'live', 'mask', 'photo']
outputFolderPath = 'Dataset/DataCollect'
confidence = 0.8
save = True
blurThreshold = 5  # Larger is more focus

debug = False
offsetPercentageW = 10
offsetPercentageH = 20
camWidth, camHeight = 640, 480
floatingPoint = 6
####################################


cap = cv2.VideoCapture(0)
cap.set(3, camWidth)
cap.set(4, camHeight)

detector = FaceDetector()
while True:
    success, img = cap.read()
    imgOut = img.copy()
    img, bboxs = detector.findFaces(img, draw=False)

    listBlur = []  # True False values indicating if the faces are blur or not
    listInfo = []  # The normalized values and the class name for the label txt file
    if bboxs:
        # bboxInfo - "id","bbox","score","center"
        for bbox in bboxs:
            x, y, w, h = bbox["bbox"]
            score = bbox["score"][0]
            # print(x, y, w, h)

            # ------  Check the score --------
            if score > confidence:

                # ------  Adding an offset to the face Detected --------
                offsetW = (offsetPercentageW / 100) * w
                x = int(x - offsetW)
                w = int(w + offsetW * 2)
                offsetH = (offsetPercentageH / 100) * h
                y = int(y - offsetH * 3)
                h = int(h + offsetH * 3.5)

                # ------  To avoid values below 0 --------
                if x < 0: x = 0
                if y < 0: y = 0
                if w < 0: w = 0
                if h < 0: h = 0

                # ------  Find Blurriness --------
                imgFace = img[y:y + h, x:x + w]
                cv2.imshow("w", imgFace)
                blurValue = int(cv2.Laplacian(imgFace, cv2.CV_64F).var())
                #if blurValue > blurThreshold:
                    #listBlur.append(True)
                #else:
                    #listBlur.append(False)

                # ------  Normalize Values  --------
                ih, iw, _ = img.shape
                xc, yc = x + w / 2, y + h / 2

                xcn, ycn = round(xc / iw, floatingPoint), round(yc / ih, floatingPoint)
                wn, hn = round(w / iw, floatingPoint), round(h / ih, floatingPoint)
                # print(xcn, ycn, wn, hn)

                # ------  To avoid values above 1 --------
                if xcn > 1: xcn = 1
                if ycn > 1: ycn = 1
                if wn > 1: wn = 1
                if hn > 1: hn = 1

            
                listInfo.append(f"{classID} {xcn} {ycn} {wn} {hn}\n")

                # ------  Drawing --------
                cv2.rectangle(imgOut, (x, y, w, h), (255, 0, 0), 3)
                cvzone.putTextRect(imgOut, f'Score: {int(score * 100)}% Blur: {blurValue}', (x, y - 0),
                                   scale=2, thickness=3)
                if debug:
                    cv2.rectangle(img, (x, y, w, h), (255, 0, 0), 3)
                    cvzone.putTextRect(img, f'Score: {int(score * 100)}% Blur: {blurValue}', (x, y - 0),
                                       scale=2, thickness=3)

        # ------  To Save --------
        if cv2.waitKey(1) & 0xFF == ord('c'):
            if blurValue > blurThreshold:
                # ------  Save Image  --------
                timeNow = time()
                timeNow = str(timeNow).split('.')
                timeNow = timeNow[0] + timeNow[1]
                cv2.imwrite(f"{outputFolderPath}/{timeNow}.jpg", img)
                info = (f"{classID} {xcn} {ycn} {wn} {hn}\n")
                # ------  Save Label Text File  --------
                f = open(f"{outputFolderPath}/{timeNow}.txt", 'a')
                f.write(info)
                f.close()
                print('OK')

    cv2.imshow("Image", imgOut)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [14]:
import os
import random
import shutil
from itertools import islice

outputFolderPath = "Dataset/SplitData"
inputFolderPath = "Dataset/DataCollect"
splitRatio = {"train": 0.7, "val": 0.2, "test": 0.1}
classes = ['device', 'live', 'mask', 'photo']

try:
    shutil.rmtree(outputFolderPath)
except OSError as e:
    os.mkdir(outputFolderPath)

# --------  Directories to Create -----------
os.makedirs(f"{outputFolderPath}/train/images", exist_ok=True)
os.makedirs(f"{outputFolderPath}/train/labels", exist_ok=True)
os.makedirs(f"{outputFolderPath}/val/images", exist_ok=True)
os.makedirs(f"{outputFolderPath}/val/labels", exist_ok=True)
os.makedirs(f"{outputFolderPath}/test/images", exist_ok=True)
os.makedirs(f"{outputFolderPath}/test/labels", exist_ok=True)

# --------  Get the Names  -----------
listNames = os.listdir(inputFolderPath)

uniqueNames = []
for name in listNames:
    uniqueNames.append(name.split('.')[0])
uniqueNames = list(set(uniqueNames))

# --------  Shuffle -----------
random.shuffle(uniqueNames)

# --------  Find the number of images for each folder -----------
lenData = len(uniqueNames)
lenTrain = int(lenData * splitRatio['train'])
lenVal = int(lenData * splitRatio['val'])
lenTest = int(lenData * splitRatio['test'])

# --------  Put remaining images in Training -----------
if lenData != lenTrain + lenTest + lenVal:
    remaining = lenData - (lenTrain + lenTest + lenVal)
    lenTrain += remaining

# --------  Split the list -----------
lengthToSplit = [lenTrain, lenVal, lenTest]
Input = iter(uniqueNames)
Output = [list(islice(Input, elem)) for elem in lengthToSplit]
print(f'Total Images:{lenData} \nSplit: {len(Output[0])} {len(Output[1])} {len(Output[2])}')

# --------  Copy the files  -----------

sequence = ['train', 'val', 'test']
print(Output)
for i,out in enumerate(Output):
    for fileName in out:
        shutil.copy(f'{inputFolderPath}/{fileName}.jpg', f'{outputFolderPath}/{sequence[i]}/images/{fileName}.jpg')
        shutil.copy(f'{inputFolderPath}/{fileName}.txt', f'{outputFolderPath}/{sequence[i]}/labels/{fileName}.txt')

print("Split Process Completed...")


# -------- Creating Data.yaml file  -----------

dataYaml = f'path: ../Data\n\
train: ../train/images\n\
val: ../val/images\n\
test: ../test/images\n\
\n\
nc: {len(classes)}\n\
names: {classes}'


f = open(f"{outputFolderPath}/data.yaml", 'a')
f.write(dataYaml)
f.close()

print("Data.yaml file Created...")

Total Images:76 
Split: 54 15 7
[['170531203752981', '17053111183172321', '17053122429810238', '17053119863003662', '17053111120518043', '17053121491809244', '17053120402027283', '1705311119001275', '1705312239844552', '1705311649644806', '17053119331481066', '1705312009189049', '17053121512339444', '17053122804601629', '17053112892998602', '17053119818393123', '170531199927506', '17053119375254445', '17053111114302812', '17053110706288836', '17053121552885492', '17053120125557055', '17053118459615152', '1705312136752091', '17053111135861125', '17053111152184908', '170531227616634', '17053119212095613', '17053121508361254', '17053111140178125', '1705311115884361', '17053111126875966', '17053112513896735', '1705312155056394', '17053122872003841', '17053111176653476', '17053113748448389', '17053122795384543', '17053113089789443', '17053119969214027', '17053122827901123', '17053111144185314', '17053119847715003', '1705311074069298', '1705311072991041', '17053119978284302', '17053121493144

In [2]:
from ultralytics import YOLO

In [3]:
model = YOLO('runs/detect/train3/weights/last.pt')
results = model.train(data='Face.v5-noise.yolov8/data.yaml', epochs=50, imgsz=320, device=[0])

New https://pypi.org/project/ultralytics/8.0.236 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.226 🚀 Python-3.11.7 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=runs/detect/train3/weights/last.pt, data=Face.v5-noise.yolov8/data.yaml, epochs=50, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_fr

train: Scanning E:\IOT\Face.v5-noise.yolov8\train\labels.cache... 7059 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7059/7059 [00:00<?, ?it/s]
val: Scanning E:\IOT\Face.v5-noise.yolov8\valid\labels... 336 images, 0 backgrounds, 0 corrupt: 100%|██████████| 336/336 [00:00<00:00, 882.98it/s]

val: New cache created: E:\IOT\Face.v5-noise.yolov8\valid\labels.cache


Plotting labels to runs\detect\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to runs\detect\train6
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50     0.841G     0.4882      0.388     0.9755          8        320: 100%|██████████| 442/442 [04:44<00:00,  1.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:06<00:00,  1.65it/s]

...

      50/50     0.847G     0.1685     0.1153     0.8038          4        320: 100%|██████████| 442/442 [04:54<00:00,  1.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:07<00:00,  1.56it/s]

                   all        336        361       0.99      0.933      0.969       0.93



50 epochs completed in 3.987 hours.
Optimizer stripped from runs\detect\train6\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train6\weights\best.pt, 6.2MB

Validating runs\detect\train6\weights\best.pt...
Ultralytics YOLOv8.0.226 🚀 Python-3.11.7 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:07<00:00,  1.56it/s]


                   all        336        361       0.99      0.933      0.969       0.93
                device        336        105      0.998      0.981       0.99      0.982
                  live        336         80      0.974      0.937      0.967      0.871
                  mask        336         79      0.986      0.924      0.959      0.942
                 photo        336         97          1       0.89       0.96      0.927
Speed: 0.2ms preprocess, 13.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\detect\train6


In [3]:
from ultralytics import YOLO
model = YOLO('antispoof.pt')
results = model.train(data='../DatasetYOLOCollection/Face YOLO Data/Face.v2-hue.yolov8/data.yaml', epochs=10, imgsz=320, device=[0])

New https://pypi.org/project/ultralytics/8.1.1 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.226 🚀 Python-3.11.7 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=antispoof.pt, data=../DatasetYOLOCollection/Dataset/SplitData/data.yaml, epochs=10, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train31, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_fra

train: Scanning E:\IOT\DatasetYOLOCollection\Dataset\SplitData\train\labels.cache... 56 images, 0 backgrounds, 0 corrupt: 100%|██████████| 56/56 [00:00<?, ?it/s]
val: Scanning E:\IOT\DatasetYOLOCollection\Dataset\SplitData\valid\labels.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]


Plotting labels to runs\detect\train31\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
